In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
import joblib

In [2]:
#Read data
df = pd.read_csv('/home/edward/Homelab/Machine_learning/Drug_recommendation_with_sentiment/Data/drugsComTest_complete.csv', sep=',')


input_c = input("Enter the medical condition: ")

#Create mask variable to define and process data to required parameters, in this case to keep all values with rating 10 as True. The mask results categorize as True or False, and keeps all True. Apply the mask to the DataFrame and save it as a new DataFrame.
mask = df["rating"] == 10
df_mask = df[mask]

mask_2 = df_mask["condition"] == input_c
df_mask = df_mask[mask_2]


#Extract specific data for independent variable within the DataFrame columns.
X = df_mask[["usefulCount", "review"]]

#Replace name with numbers, by creating labels for the independent variable "condition".
#Import Sklearn LabelEncoder to encode the independent variable "condition".
from sklearn import preprocessing

#Extract specific data for dependent variable within the DataFrame columns.
Y = df_mask[['drugName']]

#Replace name with numbers - Labels
le_drugN = preprocessing.LabelEncoder()
le_drugN.fit(Y['drugName'])
Y['drugName'] = le_drugN.transform(Y['drugName'])

/tmp/ipykernel_169151/1834454344.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y['drugName'] = le_drugN.transform(Y['drugName'])


In [3]:
#Get only the top 20 drugNames with the highest number of usefulCount.

top_20 = X.sort_values(by=['usefulCount'], ascending=False)
top_20 = top_20.head(20)

#Get the index of the top 20 drugNames.
top_20_index = top_20.index

top_20_index_list = top_20_index.tolist()

In [4]:
### DONT RUN!

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/home/edward/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def sentiment_analysis(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [6]:
#Create a new column in the DataFrame, with the sentiment analysis of the review.
from cgi import print_environ


sentiment_score = []

#Sentiment analysis for the top 20 drugNames.
for i in top_20_index_list:
    sentiment_score.append(sentiment_analysis(X['review'][i]))

print(sentiment_score)


[5, 2, 2, 2, 5, 5, 2, 5, 5, 5, 5, 1, 1, 2, 5, 5, 5, 5, 5, 5]


In [7]:
#Indexes of the highest scores in sentiment_score.
highest_score_index = [i for i, x in enumerate(sentiment_score) if x == max(sentiment_score)]

recommend_drug_index = []
recommend_drug_name = []
recommend_drug_score = []

for k in highest_score_index:
    recommend_drug_index.append(top_20_index_list[k])

for o in recommend_drug_index:
    recommend_drug_name.append(Y['drugName'][o])

#Transform the drugName numbers in recommend_drug_name back to the original names.
recommend_drug_name = le_drugN.inverse_transform(recommend_drug_name)

print(recommend_drug_name)

def recommend_drug_review():
    #get the review of the drug that the client requested.
    client_request_drug_review = X['review']

    return client_request_drug_review



['Sertraline' 'Vilazodone' 'Paroxetine' 'Pristiq' 'Desvenlafaxine'
 'Prozac' 'Wellbutrin' 'Desvenlafaxine' 'Pristiq' 'Modafinil' 'Bupropion'
 'Fetzima' 'Celexa']


In [8]:
import joblib

#Save the model
joblib.dump(recommend_drug_name, "drug_name.joblib")
joblib.dump(sentiment_score, "drug_score.joblib")
joblib.dump(recommend_drug_review(), "drug_review.joblib")

['drug_review.joblib']

In [9]:
# Create a class to store the data
class Drug:
    def __init__(self, name, score, review):
        self.name = name
        self.score = score
        self.review = review

    def __repr__(self):
        return f"{self.name}: {self.score}"
    
    def __str__(self):
        return f"{self.name}: {self.score}"
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __hash__(self):
        return hash(self.name)
    
    def __lt__(self, other):
        return self.score < other.score
    
    def __gt__(self, other):
        return self.score > other.score
    
    def __le__(self, other):
        return self.score <= other.score

    def __ge__(self, other):
        return self.score >= other.score
    
    def name(self):
        return self.name

    def score(self):
        return self.score
    
    def review(self):
        return self.review

# Create a list of drug objects
drugs = [Drug(name, score, review) for name, score, review in zip(recommend_drug_name, sentiment_score, recommend_drug_review())]

# Sort the list of drug objects
drugs.sort(reverse=True)

# Print the list of drug objects
# print(Drug("Prozac", 0))
drug_user_names = []
for i in range(len(drugs)):
    drug_user_names.append(Drug.name(drugs[i]))

print(f"These are the drugs best suited for the condition specified: \n {drug_user_names} \n" )


def user_interaction():
    input_drug = input("Please pick a drug name you would like to know more about from the list above, enter a drug name: ")

    drug_index = drugs.index(Drug(f"{input_drug}", 0, 0))

    return f"The score of {input_drug} is {Drug.score(drugs[drug_index])} and the review is {Drug.review(drugs[drug_index])} \n"

print(user_interaction())

continue_question = input("Would you like to know more about another drug? Enter 'y' for yes, 'n' for no: ")

while continue_question == "y":
    print(user_interaction())

if continue_question == "n":
    print("Thank you for using our service!")
        

These are the drugs best suited for the condition specified: 
 ['Sertraline', 'Desvenlafaxine', 'Prozac', 'Desvenlafaxine', 'Pristiq', 'Modafinil', 'Bupropion', 'Vilazodone', 'Paroxetine', 'Pristiq', 'Wellbutrin', 'Fetzima', 'Celexa'] 

The score of Sertraline is 5 and the review is "I&#039;ve tried a few antidepressants over the years (citalopram, fluoxetine, amitriptyline), but none of those helped with my depression, insomnia &amp; anxiety. My doctor suggested and changed me onto 45mg mirtazapine and this medicine has saved my life. Thankfully I have had no side effects especially the most common - weight gain, I&#039;ve actually lost alot of weight. I still have suicidal thoughts but mirtazapine has saved me." 

